# Four-node NEM Capacity Expansion model

A simple four node model of NEM 

In [ ]:
import pypsa
from pypsa.common import annuity

In [ ]:
#Create network

network = pypsa.Network(name="four-node-nem")

#Add nodes
network.add("Bus", name="NSW", x=151.20, y=-33.87)
network.add("Bus", name="QLD", x=153.03, y=-27.47)
network.add("Bus", name="SA", x=138.63, y=-34.93)
network.add("Bus", name="VIC", x=145.01, y=-37.81)

In [ ]:
m = network.plot.map(boundaries = [138, 154, -45, -25], bus_size=.1)

In [ ]:
CARRIERS = {
    "solar": "#F8E71C", #"gold",
    "wind": "#417505", #"green",
    "gas": "#F48E1B", #"orange",
    "brown_coal": "#8B572A", #brown
    "black_coal": "black",
    "load": "slategrey",
    "AC": "violet"
}

network.add(
    "Carrier",
    CARRIERS.keys(),
    color=CARRIERS.values(),
)

## Adding snapshots and loads to the buses
 
- First we need to set the "snapshots". Snapshots are the main way time is represented in the PyPSA.
- We then add load to each of the buses.

In this case, we by loading a data frame of load data, with a datetime index. We use the datetime index of this to create the snapshots. And then add the active power demand (`p_set`, in MW) at each time step from the dataframe to loads at each bus.


In [ ]:
#load the data
df_load = pd.read_csv("demand.csv", index_col=0, parse_dates=True)
display(df_load)

In [ ]:
# Set snapshots
network.set_snapshots(df_load.index)

# Add Load component
network.add("Load", 
            ["VIC", "SA", "NSW", "QLD"],
            suffix="_load", 
            bus=["VIC", "SA", "NSW", "QLD"],
            p_set=df_load[["VIC", "SA", "NSW", "QLD"]], 
            carrier="load")

In [ ]:
display(network)


In [ ]:
display(network.loads)

## Adding generation to the buses. 

We then add generation to the buses. 


In [ ]:
## Add brown coal 

network.add(
    "Generator",
    ["VIC"],
    suffix="_brown",
    bus=["VIC"],
    p_nom=3000,
    marginal_cost=20,
    carrier="brown_coal",
    ramp_limit_up=0.1,
    ramp_limit_down=0.2,
    overwrite=True
)

## Add black coal 

network.add(
    "Generator",
    ["NSW", "QLD"],
    suffix="_black",
    bus=["NSW", "QLD"],
    p_nom=4000,
    marginal_cost=60,
    carrier="black_coal",
    ramp_limit_up=0.1,
    ramp_limit_down=0.2,
    overwrite=True
)


In [ ]:
display(network.generators.T)

## New Generation

**For potential new generation, we set p_nom_extendable=True**

(This is necessary for capacity expansion)

For variable renewable resources, we pass the capacity factors as p_max_pu (per-unit (p.u.) time series that defines the maximum available output of a generator relative to its nominal capacity (p_nom) at each snapshot)



In [ ]:
#Load solar data
df_solar = pd.read_csv("solar.csv", index_col=0, parse_dates=True)
df_solar.head()

In [ ]:
network.add(
    "Generator",
    ["VIC", "SA", "NSW", "QLD"],
    suffix="_solar",
    bus=["VIC", "SA", "NSW", "QLD"],
    p_max_pu=df_solar[["VIC", "SA", "NSW", "QLD"]],
    p_nom_extendable=True,
    capital_cost=annuity(0.05, 30) * 1_000_000,
    carrier="solar")


In [ ]:
#Load wind data
df_wind = pd.read_csv("wind.csv", index_col=0, parse_dates=True)
df_wind.head()

network.add(
    "Generator",
    ["VIC", "SA", "NSW", "QLD"],
    suffix="_wind",
    bus=["VIC", "SA", "NSW", "QLD"],
    p_max_pu=df_wind[["VIC", "SA", "NSW", "QLD"]],
    p_nom_extendable=True,
    capital_cost=annuity(0.05, 30) * 3_000_000,
    carrier="wind")

In [ ]:
## Add gas generation
network.add(
    "Generator",
    ["VIC", "SA", "NSW", "QLD"],
    suffix="_gas",
    bus=["VIC", "SA", "NSW", "QLD"],
    p_nom_extendable=True,
    marginal_cost=120,
    capital_cost=annuity(0.05, 30) * 2_000_000,
    carrier="gas"
)


## Adding links between the nodes

We then add "links" between the nodes, to allow for transmission of power between the buses. 

In [ ]:
network.add(
        "Link",
        f"VIC-SA",
        bus0="VIC",
        bus1="SA",
        p_nom=1000,
        carrier="AC",
        p_min_pu=-1,  # bidirectional
        overwrite=True
    )

network.add(
        "Link",
        f"VIC-NSW",
        bus0="VIC",
        bus1="NSW",
        p_nom=2500,
        carrier="AC",
        p_min_pu=-1,  # bidirectional
        overwrite=True
    )

network.add(
        "Link",
        f"NSW-QLD",
        bus0="NSW",
        bus1="QLD",
        p_nom=1500,
        carrier="AC",
        p_min_pu=-1,  # bidirectional
        overwrite=True
    )


## Running the optimisation

In [ ]:
network.optimize()



## Exploring the results

In [ ]:
network.statistics()


In [ ]:
network.statistics.energy_balance().div(1e6).round(2).sort_values()

In [ ]:
average_cost = (
    (network.statistics.capex().sum() + network.statistics.opex().sum())
    / network.loads_t.p_set.sum().sum()
)
display(f"Average cost: {average_cost:.2f} AUD/MWh")

In [ ]:
network.buses_t.marginal_price.mean()


In [ ]:
import cartopy.crs as ccrs
crs = ccrs.PlateCarree()
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={"projection": crs})

# Use the energy balance statistics to prepare the bus sizes and plot the network
bus_size = network.statistics.energy_balance(groupby=["bus", "carrier"]).droplevel(
    "component")

d = network.plot(ax=ax, bus_size=bus_size / 4e7, margin=0.75, bus_split_circle=True, boundaries = [135, 160, -45, -20])








In [ ]:
network.statistics.energy_balance.iplot()
